# Loja de Departamento Tech

### Criando pré Dataset com Produto, Categoria e Preço

In [1]:
import pandas as pd
import numpy as np

dados = [['Teclado','Informática', '140'], ['Mouse','Informática', '90'], ['Placa de video', 'Informática', '3500'], ['Monitor','Informática', '1900'], ['Iphone 16', 'Smartphone', '7000'],
            ['Aspirador Robô','Eletrodomésticos', '1200'], ['Televisão','Televisores', '4600'], ['Notebook', 'Informática', '5500'], ['Air Fryer', 'Eletrodomésticos', '700'], ['Playstation 5','Games', '3000'],
            ['Adega', 'Eletrodomésticos', '800'], ['Alexa', 'Assistentes virtual', '400'], ['Óculos VR', 'Smart Accessories'], ['Smart watch', 'Smart Accessories', '2000'], ['Smart ring', 'Smart Accessories', '1300'],
            ['Google Home', 'Assistentes virtual', '450'], ['Xbox Series S', 'Games', '4500'], ['Playstation 5', 'Games', '3200'], ['Nintendo Switch', 'Games', '2600'], ['ROG Ally', 'Games', '3700'],
            ['Playstation Portal', 'Games', '2300'], ['DJI mini 3', 'Drones', '5300'], ['Iphone 16 pro', 'Smartphone', '9500'], ['Iphone 16 pro max','Smartphone', '1200'], ['Galaxy s24', 'Smartphone', '3200'],
            ['Galaxy s24 plus', 'Smartphone', '4500'], ['Galaxy s24 Ultra', 'Smartphone', '6700']]

df = pd.DataFrame(dados, columns = ['Produto', 'Categoria', 'Preço'])
df.head()

,Produto,Categoria,Preço
0,Teclado,Informática,140
1,Mouse,Informática,90
2,Placa de video,Informática,3500
3,Monitor,Informática,1900
4,Iphone 16,Smartphone,7000


### Criando Dataset completo, com data, IDs e tipo correto dos dados

In [2]:
import random
# Expandindo para 100 registros
num_registros = 100
produtos = df.to_dict('records') #Convertendo para dicionário, cada produto é um registro
registros = [random.choice(produtos) for _ in range(num_registros)]

# Adicionando IDs e datas sequenciais
datas = pd.date_range(start='2023-01-01', end='2023-12-31') #gerando datas entre janeiro e dezembro de 2023
datas_aleatorias = [random.choice(datas) for _ in range(num_registros)] #numero de datas = numero de registros
for i, registro in enumerate(registros): #adcionanado ids, data e quantidade
    registro['ID'] = i + 1000
    registro['Data'] = datas_aleatorias[i]
    registro['Quantidade'] = random.randint(12, 53)

df = pd.DataFrame(registros)
df

,Produto,Categoria,Preço,ID,Data,Quantidade
0,Adega,Eletrodomésticos,800,1097,2023-09-12,24
1,ROG Ally,Games,3700,1089,2023-03-21,40
2,Teclado,Informática,140,1086,2023-04-22,45
3,Adega,Eletrodomésticos,800,1097,2023-09-12,24
4,Playstation 5,Games,3000,1069,2023-04-14,34
...,...,...,...,...,...,...
95,Placa de video,Informática,3500,1095,2023-09-04,33
96,Galaxy s24 plus,Smartphone,4500,1096,2023-11-13,53
97,Adega,Eletrodomésticos,800,1097,2023-09-12,24
98,Mouse,Informática,90,1098,2023-07-23,20


### Transformando os tipos dos dados

In [3]:
df['Preço'] = pd.to_numeric(df['Preço'], errors = 'coerce')
df['Quantidade'] = pd.to_numeric(df['Quantidade'], errors = 'coerce')
df['Data'] = pd.to_datetime(df['Data'])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Produto     100 non-null    object        
 1   Categoria   100 non-null    object        
 2   Preço       98 non-null     float64       
 3   ID          100 non-null    int64         
 4   Data        100 non-null    datetime64[ns]
 5   Quantidade  100 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 4.8+ KB


### Conferindo média dos preços por categorias para ingestão dos dados
- Números com apenas duas casas decimais para facilitar a ingestão dos dados.
- Temos que conferir os valores antes de adicionar dados Nans

In [4]:
df.groupby('Categoria')['Preço'].mean().round(2)

,Preço
Categoria,
Assistentes virtual,421.43
Drones,5300.00
Eletrodomésticos,947.83
Games,2988.24
Informática,1941.30
Smart Accessories,1475.00
Smartphone,4952.63


### Adcionando valores faltantes e duplicadatas
- Temos que adicionar NaNs e duplicatas depois de criar e converter os tipos dos dados

In [5]:
for _ in range(47): #adicionando 47 valores faltantes na coluna de preço
  df.loc[random.choice(df.index), 'Preço'] = np.nan

### Adicionando duplicatas

In [6]:
df = pd.concat([df, df.sample(9)]) #adicionando 9 duplicatas

### Conferindo valores faltantes

In [7]:
df.isna().sum()

,0
Produto,0
Categoria,0
Preço,42
ID,0
Data,0
Quantidade,0


### Conferindo quais categorias tem o preço NaN para adicionar a média do preço de acordo com sua categoria

In [8]:
df[df['Preço'].isna()]['Categoria'].unique()

array(['Eletrodomésticos', 'Informática', 'Games', 'Smart Accessories',
       'Smartphone', 'Assistentes virtual', 'Drones'], dtype=object)

### Preenchendo dados faltantes

In [9]:
media_precos = {'Smart Accessories':1475.00, 'Informática': 1941.30, 'Games': 2988.24,
       'Assistentes virtual': 421.43, 'Smartphone': 4952.63, 'Eletrodomésticos': 947.83, 'Drones': 5300.00}

df['Preço'] = df.apply(lambda row: media_precos.get(row['Categoria'], 0) if pd.isna(row['Preço']) else row['Preço'], axis=1) #Adicionando a média dos preços de acordo com a categoria
df['Categoria'] = df['Categoria'].fillna('Não informado') #adcionando "não informado" para valores nulos
df['Produto'] = df['Produto'].fillna('Não informado') #adcionando "não informado" para valores nulos

### Conferindo o novo Dataset

In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 109 entries, 0 to 59
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype         
---  ------      --------------  -----         
 0   Produto     109 non-null    object        
 1   Categoria   109 non-null    object        
 2   Preço       109 non-null    float64       
 3   ID          109 non-null    int64         
 4   Data        109 non-null    datetime64[ns]
 5   Quantidade  109 non-null    int64         
dtypes: datetime64[ns](1), float64(1), int64(2), object(2)
memory usage: 6.0+ KB


In [11]:
df.isnull().sum()

,0
Produto,0
Categoria,0
Preço,0
ID,0
Data,0
Quantidade,0


In [12]:
df.isna().sum()

,0
Produto,0
Categoria,0
Preço,0
ID,0
Data,0
Quantidade,0


### Removendo duplicatas

In [13]:
df.drop_duplicates(inplace = True)

### Salvando o dataset limpo

In [14]:
df.to_csv('data_clean_csv')

## Análises
### Adicionando coluna de vendas totais (Preço * Quantidade)

In [15]:
df['Vendas_totais'] = df['Preço'] * df['Quantidade']
df

,Produto,Categoria,Preço,ID,Data,Quantidade,Vendas_totais
0,Adega,Eletrodomésticos,947.83,1097,2023-09-12,24,22747.92
1,ROG Ally,Games,3700.00,1089,2023-03-21,40,148000.00
2,Teclado,Informática,1941.30,1086,2023-04-22,45,87358.50
4,Playstation 5,Games,2988.24,1069,2023-04-14,34,101600.16
5,Óculos VR,Smart Accessories,1475.00,1049,2023-06-13,39,57525.00
6,Adega,Eletrodomésticos,800.00,1097,2023-09-12,24,19200.00
7,Placa de video,Informática,3500.00,1095,2023-09-04,33,115500.00
8,Nintendo Switch,Games,2600.00,1093,2023-10-23,38,98800.00
10,Iphone 16,Smartphone,4952.63,1044,2023-02-06,46,227820.98
11,Nintendo Switch,Games,2988.24,1093,2023-10-23,38,113553.12


- Identifique o produto com o maior número de vendas totais:
- iPhone 16 é o nosso produto mais vendido

In [16]:
top_produto = df.groupby('Produto')['Vendas_totais'].sum().sort_values(ascending = False).head(10)
top_produto

,Vendas_totais
Produto,
Iphone 16,549820.98
DJI mini 3,280900.00
ROG Ally,267529.60
Galaxy s24 plus,262489.39
Playstation 5,254800.16
Galaxy s24,228273.64
Nintendo Switch,212353.12
Notebook,208356.40
Monitor,203588.90


## Análise exploratória dos dados de venda
- Crie um gráfico de linha mostrando a tendência de vendas ao longo do tempo, analisando os dados mensalmente:

In [20]:
import plotly.express as px

#Criando a coluna do mês
df['Mes'] = df['Data'].dt.to_period('M').astype(str)

#vendas mensais
mensal = df.groupby('Mes')['Vendas_totais'].sum().reset_index()

#grafico
fig = px.line(mensal, x = 'Mes', y = 'Vendas_totais', title = 'Tendência de vendas mensais ao longo do tempo', markers = True)
fig.show()

### Gráficos para ajudar nossos insights e tomadas de decisão

In [28]:
top_produto = df.groupby('Produto')['Vendas_totais'].sum().reset_index().sort_values(by='Vendas_totais', ascending=False).head(10)

fig = px.bar(top_produto, x = 'Produto',  y = 'Vendas_totais', title = 'Top 10 produtos mais vendidos', color = 'Produto')
fig.show()

In [50]:
fig.write_html('top_produtos.html')

In [51]:
top_produto = df.groupby('Categoria')['Vendas_totais'].sum().reset_index().sort_values(by='Vendas_totais', ascending=False).head(10)

fig2 = px.bar(top_produto, x = 'Categoria',  y = 'Vendas_totais', title = 'Top 7 categorias mais vendidas', color = 'Categoria')
fig2.show()

In [52]:
fig2.write_html('top_categorias.html')

In [53]:
top_smartphones = df[df['Categoria'] == 'Smartphone'].groupby('Produto')['Vendas_totais'].sum().reset_index().sort_values(by='Vendas_totais', ascending=False).head(5)

fig3 = px.bar(top_smartphones, x = 'Produto',  y = 'Vendas_totais', title = 'Top 5 Smartphones mais vendidos', color = 'Produto')
fig3.show()

In [54]:
fig3.write_html('top_smartphones.html')

In [55]:
top_games = df[df['Categoria'] == 'Games'].groupby('Produto')['Vendas_totais'].sum().reset_index().sort_values(by='Vendas_totais', ascending=False).head(4)

fig4 = px.bar(top_games, x = 'Produto',  y = 'Vendas_totais', title = 'Top 4 games mais vendidos', color = 'Produto')
fig4.show()

In [56]:
fig4.write_html('top_games.html')

In [57]:
top_informatica = df[df['Categoria'] == 'Informática'].groupby('Produto')['Vendas_totais'].sum().reset_index().sort_values(by='Vendas_totais', ascending=False).head(5)

fig5 = px.bar(top_informatica, x = 'Produto',  y = 'Vendas_totais', title = 'Top 5 produtos de informática mais vendidos', color = 'Produto')
fig5.show()

In [58]:
fig5.write_html('top_informatica.html')

## Identifique e descreva pelo menos dois padrões ou insights interessantes que você observou nos dados:
- Nossa loja está com um tendência ruim para as vendas mensais. No começo do ano, podemos notar que as vendas estão ótimas e, por consequência, o lucro também. Mas, no mês de março, nossa loja sofre uma queda grande, gerando grandes impactos. Durante praticamente todo ano, nossa loja fica em queda, subindo um pouco em relação ao mês anterior, mas em queda em relação aos primeiros meses do ano. Podemos ver que em setembro, temos uma alta maior em relação aos meses passados, mas mesmo assim não o suficiente para ultrapassar nosso melhor mês.
#### Em que podemos melhorar:
- De acordo com nossos gráficos gerados, as 3 categorias que mais geram renda para nossa loja são: Smartphones, Games e Informática. Com isso, poderiamos aumentar nosso nível tanto de publicidade quanto de estoque dessas categorias. Tendo em vista os gráficos, os produtos iPhone 16, ROG Ally e Notebook são os mais vendidos das categorias mais vendidas, ou seja, podemos direcionar melhor nossas publicidades para o público alvo desses produtos para que possamos ter melhores vendas e resultados. Em relação aos meses em que as vendas estão em quedas significativas, poderiamos planejar um período de desconto para certas categorias (por exemplo: "black weekend de drones"), isso atrairá clientes procurando boas ofertas, aumentando nossas vendas em meses de queda.


# Parte 2: Consultas SQL
#### Conectando e utilizando nosso DataFrame

In [ ]:
import sqlite3

connect = sqlite3.connect('loja_departamento_tech.db')
cursor = connect.cursor()
df.to_sql('dados', connect, if_exists = 'replace', index = False)

#### Consulta 1:
- Listar o nome do produto, categoria e a soma total de vendas (Quantidade * Preço) para cada produto. Ordene o resultado pelo valor total de vendas em ordem decrescente

In [43]:
primeira = """
SELECT
    Produto,
    Categoria,
    SUM(Vendas_totais) AS Total_Vendas
FROM
    dados
GROUP BY
    Produto, Categoria
ORDER BY
    Total_Vendas DESC;
"""

cursor.execute(primeira)
result_1 = cursor.fetchall()

In [47]:
result_1

[('Iphone 16', 'Smartphone', 549820.98),
 ('DJI mini 3', 'Drones', 280900.0),
 ('ROG Ally', 'Games', 267529.6),
 ('Galaxy s24 plus', 'Smartphone', 262489.39),
 ('Playstation 5', 'Games', 254800.15999999997),
 ('Galaxy s24', 'Smartphone', 228273.64),
 ('Nintendo Switch', 'Games', 212353.12),
 ('Notebook', 'Informática', 208356.4),
 ('Monitor', 'Informática', 203588.9),
 ('Galaxy s24 Ultra', 'Smartphone', 198094.71000000002),
 ('Placa de video', 'Informática', 179562.9),
 ('Playstation Portal', 'Games', 121900.0),
 ('Teclado', 'Informática', 93658.5),
 ('Aspirador Robô', 'Eletrodomésticos', 79469.70999999999),
 ('Iphone 16 pro', 'Smartphone', 64384.19),
 ('Óculos VR', 'Smart Accessories', 57525.0),
 ('Smart ring', 'Smart Accessories', 45500.0),
 ('Adega', 'Eletrodomésticos', 41947.92),
 ('Mouse', 'Informática', 40626.0),
 ('Smart watch', 'Smart Accessories', 38350.0),
 ('Iphone 16 pro max', 'Smartphone', 31200.0),
 ('Alexa', 'Assistentes virtual', 25464.33),
 ('Air Fryer', 'Eletrodomésti

#### Consulta 2:
- Identificar os produtos que venderam menos no mês de junho de 2024.
Salve suas consultas e a explicação da lógica em um arquivo chamado consultas_sql.sql.
-----
**----Não entendi o porquê de ser junho de 2024 sendo que vocês pediram para montar o dataset no ano de 2023, então fiz para 2023----**

In [44]:
segunda = """
SELECT
    Produto,
    Categoria,
    SUM(Vendas_totais) AS Total_Vendas
FROM
    dados
WHERE
    Mes = '2023-06'
GROUP BY
    Produto, Categoria
ORDER BY
    Total_Vendas ASC;
"""

cursor.execute(segunda)
result_2 = cursor.fetchall()

In [48]:
result_2

[('Óculos VR', 'Smart Accessories', 57525.0),
 ('Iphone 16 pro', 'Smartphone', 64384.19),
 ('DJI mini 3', 'Drones', 280900.0)]

#### Salvando as consultas no arquivo

In [45]:
with open('consultas_sql.sql', 'w') as f:
    f.write("-- Primeira consulta\n")
    f.write(primeira + "\n\n")
    f.write("-- Segunda consulta\n")
    f.write(segunda + "\n")